# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [19]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\iiats\Anaconda3\envs\Python36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [20]:
df_receipt["sales_epoch"] = pd.to_datetime(df_receipt["sales_epoch"], unit='s').dt.strftime('%d')
df_receipt[['receipt_no', 'receipt_sub_no', 'sales_epoch']].head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [21]:
df_receipt = pd.read_csv("./data/receipt.csv")

In [22]:
# 顧客IDが"Z"から始まるのものは非会員を表すため、除外
# df = df_receipt.query("not customer_id.str.startswith("Z")", engine="python") --> エラー
# query() の中は '' じゃなかいけないのか。
df = df_receipt.query('not customer_id.str.startswith("Z")', engine="python")
df2 = df[['customer_id', 'amount']].groupby('customer_id').sum().reset_index()
df2['threshhold'] = df2['amount'].apply(lambda x: 1 if x > 2000 else 0)
df2.head(10)

,customer_id,amount,threshhold
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


In [27]:
df = df_receipt.query('not customer_id.str.startswith("Z")', engine="python")
df2 = df[['customer_id', 'amount']].groupby('customer_id').sum().reset_index()

# 関数を定義して apply() に入れる
def f(x):
    if x > 2000:
        return 1
    else:
        return 0

df2['threshhold'] = df2['amount'].apply(f) # apply() に入れる関数は()を書かなくてよい
df2.head(10)

,customer_id,amount,threshhold
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [38]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [35]:
df = df_customer[['customer_id', 'postal_cd']]
df['threshhold'] = df['postal_cd'].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)
pd.merge(df, df_receipt, how='inner', on='customer_id').groupby('threshhold').agg({'customer_id':'nunique'}).reset_index()

C:\Users\iiats\Anaconda3\envs\Python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,threshhold,customer_id
0,0,3906
1,1,4400


In [36]:
df = df_customer[['customer_id', 'postal_cd']].copy() # copy() を入れるとエラーがなくなる
df['threshhold'] = df['postal_cd'].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)
pd.merge(df, df_receipt, how='inner', on='customer_id').groupby('threshhold').agg({'customer_id':'nunique'}).reset_index()

,threshhold,customer_id
0,0,3906
1,1,4400


---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [42]:
# map() で辞書型をいれて置き換える
# https://note.nkmk.me/python-pandas-map-replace/
df_customer['address_code'] = df_customer['address'].str[0:3].map({'埼玉県': '11','千葉県':'12', '東京都':'13', '神奈川':'14'})
df_customer[['customer_id', 'address', 'address_code']].head(10)

,customer_id,address,address_code
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [43]:
df = df_receipt[['customer_id', 'amount']].groupby('customer_id').sum().reset_index()
df.head()

,customer_id,amount
0,CS001113000004,1298
1,CS001114000005,626
2,CS001115000010,3044
3,CS001205000004,1988
4,CS001205000006,3337


In [48]:
# np.percentile() の情報　https://analytics-note.xyz/programming/numpy-percentile/
percentile25 = np.percentile(df['amount'], 25)
percentile50 = np.percentile(df['amount'], 50)
percentile75 = np.percentile(df['amount'], 75)

print('percentile25:', percentile25)
print('percentile50:', percentile50)
print('percentile75:', percentile75)

# 解答例にあるように、np.quantile()を使うと以下のように書く
# percentile25 = np.quantile(df['amount'], 0.25) 　<-- 25 でなく 0.25 とする

percentile25: 548.5
percentile50: 1478.0
percentile75: 3651.0


In [49]:
def divide_percentile(x):
    if x < percentile25:
        return 1
    elif percentile25 <= x < percentile50:
        return 2
    elif percentile50 <= x < percentile75:
        return 3
    elif percentile75 <= x:
        return 4

In [51]:
df['percentile'] = df['amount'].apply(divide_percentile)
df.head(10)

# 解答例にあるように以下のようにlambda関数を使ってもよいが、上記の方が簡潔
# df['percentile'] = df['amount'].apply(lambda x: divide_percentile(x))

,customer_id,amount,percentile
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


In [60]:
# 公式解答のコード例2
df_temp = df_receipt.groupby('customer_id')[['amount']].sum()
df_temp.head()

,amount
customer_id,
CS001113000004,1298
CS001114000005,626
CS001115000010,3044
CS001205000004,1988
CS001205000006,3337


In [64]:
# pandas の Binning 処理について、https://note.nkmk.me/python-pandas-cut-qcut-binning/
# pandas はどれだけ機能があるんだ！
df_temp['quantile'], bins = pd.qcut(df_receipt.groupby('customer_id')['amount'].sum(), 4, retbins=True)
df_temp.head()

,amount,quantile
customer_id,,
CS001113000004,1298,"(548.5, 1478.0]"
CS001114000005,626,"(548.5, 1478.0]"
CS001115000010,3044,"(1478.0, 3651.0]"
CS001205000004,1988,"(1478.0, 3651.0]"
CS001205000006,3337,"(1478.0, 3651.0]"


In [65]:
print('quantiles:', bins)

quantiles: [7.0000000e+01 5.4850000e+02 1.4780000e+03 3.6510000e+03 1.2395003e+07]
